In [41]:
# 由使用者輸入任意數量的公司代碼

cos = []

while True:
    co = input('請輸入公司代碼：（按Enter結束）')
    if co == '':
        break
    else:
        cos.append(co)

cos

['2330', '2302', '2303', '2329']

In [46]:
from datetime import datetime

year = int(datetime.now().strftime('%Y'))-1911

month = int(datetime.now().strftime('%m'))-1
#公司代號、公司名稱
api_url_1 = 'https://mops.twse.com.tw/mops/web/ajax_autoComplete'

api_url_2 = 'https://mops.twse.com.tw/mops/web/ajax_t05st10_ifrs'

Id = []
name = []
money = []
ty = []
ly = []

# requests method不是get，要用post
import requests
from bs4 import BeautifulSoup

for i in range(len(cos)):
    stk = cos[i]
    form_data_1 = {
    'encodeURIComponent': 1,
    'step': 1,
    'firstin': 1,
    'off': 1,
    'keyword4': None,
    'code1': None,
    'TYPEK2': None,
    'checkbtn': None,
    'queryName': stk ,
    'inpuType': stk ,
    'TYPEK': all,
    'isnew': True,
    'co_id': stk ,
    'year':1, 
    'month': 1}

    resp = requests.post(api_url_1, form_data_1)
    soup=BeautifulSoup(resp.text, 'lxml')
    
    id_name = soup.find_all('li')[1].text.split(' ')

    Id.append(id_name[0])
    name.append(id_name[1])

    form_data_2 = {
    'encodeURIComponent': 1,
    'step': 1,
    'firstin': 1,
    'off': 1,
    'keyword4': None,
    'code1': None,
    'TYPEK2': None,
    'checkbtn': None,
    'queryName': stk ,
    'inpuType': stk ,
    'TYPEK': all,
    'isnew': True,
    'co_id': stk ,
    'year':year, 
    'month': month}
    
    resp = requests.post(api_url_2, form_data_2)
    soup=BeautifulSoup(resp.text, 'lxml')

    money.append(soup.find('td',{'class':'odd'}).text.split(' ')[-1])

    even = soup.find_all('td',{'class':'even'})
    ty.append(even[1].text.split(' ')[-1])

    ly.append(even[3].text.split(' ')[-1])






欄位：公司代號、公司名稱、本月營收金額、增減百分比（本月）、增減百分比（累計）

In [48]:
import pandas as pd

d = {'公司代號':Id,
        '公司名稱':name,
        '本月營收金額':money,
        '增減百分比（本月）':ty,
        '增減百分比（累計）':ly
}

df = pd.DataFrame(d)
df

,公司代號,公司名稱,本月營收金額,增減百分比（本月）,增減百分比（累計）
0,2330,台積電,"243,202,811",15.66,-3.74
1,2302,麗正,"50,480",-41.01,-15.71
2,2303,聯電,"19,191,382",-21.17,-20.60
3,2329,華泰,"1,624,056",35.81,3.40


In [49]:
# 用今天的日期來做為檔名

from datetime import datetime

date=datetime.now().strftime('%Y-%m-%d')  # m要用小寫字，因大本字是指分鐘

df.to_csv(f'公司最新營收資料_{date}.csv',encoding='utf-8-sig')